In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/student-performance-prediction-using-machine-learn/sample_submission.csv
/kaggle/input/student-performance-prediction-using-machine-learn/train.csv
/kaggle/input/student-performance-prediction-using-machine-learn/test.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler , OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

sample_submission = pd.read_csv("/kaggle/input/student-performance-prediction-using-machine-learn/sample_submission.csv")
train_data = pd.read_csv("/kaggle/input/student-performance-prediction-using-machine-learn/train.csv")
test_data = pd.read_csv("/kaggle/input/student-performance-prediction-using-machine-learn/test.csv")

In [3]:
!pip install ydata-profiling

In [5]:
from ydata_profiling import ProfileReport

profile=ProfileReport(train_data,title="Student train data analysis")

profile.to_file("/kaggle/working/train_data_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 15/15 [00:00<00:00, 72.60it/s][A


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   student_id            10000 non-null  int64  
 1   gender                10000 non-null  object 
 2   attendance_rate       10000 non-null  float64
 3   study_hours_per_week  9263 non-null   float64
 4   internet_access       10000 non-null  object 
 5   parental_education    10000 non-null  object 
 6   family_income         9350 non-null   float64
 7   extra_activities      6979 non-null   object 
 8   previous_grade        10000 non-null  float64
 9   test_anxiety_level    10000 non-null  float64
 10  sleep_hours           9392 non-null   float64
 11  mobile_usage_hours    9361 non-null   float64
 12  part_time_job         10000 non-null  object 
 13  notes_quality_score   10000 non-null  float64
 14  final_score           9263 non-null   float64
dtypes: float64(9), int64

In [5]:
train_data.head(5)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,student_id,gender,attendance_rate,study_hours_per_week,internet_access,parental_education,family_income,extra_activities,previous_grade,test_anxiety_level,sleep_hours,mobile_usage_hours,part_time_job,notes_quality_score,final_score
0,1,Male,70.12,8.92,Yes,Master,4171.02,Sports,61.68,9.46,9.71,NaN,Yes,1.03,19.93
1,2,Other,73.75,10.04,No,HighSchool,NaN,Sports,81.30,4.46,6.40,1.24,No,4.00,26.22
2,3,Female,88.89,6.84,Yes,HighSchool,2645.77,Volunteer,49.10,2.58,9.60,3.55,No,8.86,22.65
3,4,Female,73.26,7.55,Yes,Bachelor,1593.86,Sports,62.52,6.82,8.03,2.53,No,5.26,20.38
4,5,Male,96.13,14.17,No,Bachelor,4284.55,Music,77.76,6.02,7.37,3.51,No,6.33,24.85


In [6]:
# Drop values where final score in train data is null
train_data=train_data.dropna(subset=['final_score'])

In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9263 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   student_id            9263 non-null   int64  
 1   gender                9263 non-null   object 
 2   attendance_rate       9263 non-null   float64
 3   study_hours_per_week  9263 non-null   float64
 4   internet_access       9263 non-null   object 
 5   parental_education    9263 non-null   object 
 6   family_income         8653 non-null   float64
 7   extra_activities      6481 non-null   object 
 8   previous_grade        9263 non-null   float64
 9   test_anxiety_level    9263 non-null   float64
 10  sleep_hours           8690 non-null   float64
 11  mobile_usage_hours    8665 non-null   float64
 12  part_time_job         9263 non-null   object 
 13  notes_quality_score   9263 non-null   float64
 14  final_score           9263 non-null   float64
dtypes: float64(9), int64(1), o

In [8]:
# Separate Features (X) and Target (y)

y = train_data['final_score']
X = train_data.drop(columns=['final_score','student_id'])

In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9263 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gender                9263 non-null   object 
 1   attendance_rate       9263 non-null   float64
 2   study_hours_per_week  9263 non-null   float64
 3   internet_access       9263 non-null   object 
 4   parental_education    9263 non-null   object 
 5   family_income         8653 non-null   float64
 6   extra_activities      6481 non-null   object 
 7   previous_grade        9263 non-null   float64
 8   test_anxiety_level    9263 non-null   float64
 9   sleep_hours           8690 non-null   float64
 10  mobile_usage_hours    8665 non-null   float64
 11  part_time_job         9263 non-null   object 
 12  notes_quality_score   9263 non-null   float64
dtypes: float64(8), object(5)
memory usage: 1013.1+ KB


In [10]:
y.info()

<class 'pandas.core.series.Series'>
Index: 9263 entries, 0 to 9999
Series name: final_score
Non-Null Count  Dtype  
--------------  -----  
9263 non-null   float64
dtypes: float64(1)
memory usage: 144.7 KB


In [11]:
categorical_features=['gender','internet_access','parental_education','extra_activities','part_time_job']

numerical_features=['attendance_rate', 'study_hours_per_week', 'family_income', 
                    'previous_grade', 'test_anxiety_level', 'sleep_hours', 
                    'mobile_usage_hours', 'notes_quality_score']

In [12]:
# Create the preprocessing pipelines for each type

from sklearn.linear_model import LinearRegression

numeric_transformer=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

categorical_transformer=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('onehot',OneHotEncoder(handle_unknown='ignore'))
    ]
)

preprocessor=ColumnTransformer(
    transformers=[
        ('num',numeric_transformer,numerical_features),
        ('cat',categorical_transformer,categorical_features)
    ]
)

model_pipeline=Pipeline(
    steps=[
        ('preprocessor',preprocessor),
        ('regressor',LinearRegression())
    ]
)



In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [14]:
model_pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['attendance_rate',
                                                   'study_hours_per_week',
                                                   'family_income',
                                                   'previous_grade',
                                                   'test_anxiety_level',
                                                   'sleep_hours',
                                                   'mobile_usage_hours',
                                                   'notes_quality_score']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['gender', 'internet_access',
                                                   'parental_education',
                                                   'extra_activities',
                                                   'part_time_job'])])),
                ('regressor', LinearRegression())])

In [15]:
print("Model Score:", model_pipeline.score(X_test, y_test))

Model Score: 0.5050843780129125


In [16]:
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. Get predictions from your test set
predictions = model_pipeline.predict(X_test)

# 2. Calculate the Mean Squared Error (MSE)
mse = mean_squared_error(y_test, predictions)

# 3. Take the square root to get the RMSE
rmse = np.sqrt(mse)

print(f"My R-squared is: {model_pipeline.score(X_test, y_test)}")
print(f"My RMSE is: {rmse}")

My R-squared is: 0.5050843780129125
My RMSE is: 2.511030466544276


In [17]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   student_id   2000 non-null   int64  
 1   final_score  2000 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 31.4 KB


In [18]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   student_id            2000 non-null   int64  
 1   gender                2000 non-null   object 
 2   attendance_rate       2000 non-null   float64
 3   study_hours_per_week  1844 non-null   float64
 4   internet_access       2000 non-null   object 
 5   parental_education    2000 non-null   object 
 6   family_income         1824 non-null   float64
 7   extra_activities      1393 non-null   object 
 8   previous_grade        2000 non-null   float64
 9   test_anxiety_level    2000 non-null   float64
 10  sleep_hours           1897 non-null   float64
 11  mobile_usage_hours    1813 non-null   float64
 12  part_time_job         2000 non-null   object 
 13  notes_quality_score   2000 non-null   float64
dtypes: float64(8), int64(1), object(5)
memory usage: 218.9+ KB


In [19]:
test_student_ids=test_data['student_id']

In [20]:
submission_test_final=test_data.drop(columns=['student_id'])

In [21]:
final_predictions = model_pipeline.predict(submission_test_final)

In [22]:
final_predictions_range=np.clip(final_predictions,0,100)

In [23]:
final_predictions_range

array([28.9088592 , 30.79275004, 21.63627356, ..., 25.71562478,
       29.93460231, 25.46950333])

In [24]:
# Create a new DataFrame with the correct columns and order
submission_df = pd.DataFrame({
    'student_id': test_student_ids,
    'final_score': final_predictions_range
})

# Save the file. index=False is critical!
submission_df.to_csv('submission.csv', index=False)

print("Submission file created successfully!")

Submission file created successfully!
